In [21]:
# Multiple Knapsacks (or containers as bins)
# Optimal way to pack items into five bins

# We have a collection of items of varying weights and values. 
#The problem is to pack a subset of the items into five bins, each of which has a maximum capacity of 100, so that the total packed value is a maximum.

# Import the libraries

import docplex.mp

# first import the Model class from docplex.mp
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr

In [22]:
#Create the data
#The data includes:
# weights: A vector containing the weights of the items.
# values: A vector containing the values of the items.
# capacities: A vector containing the capacities of the bins.
def create_data_model(num_weights, num_bins):
    import random
    data = {}
    weights = [random.randint(24,48) for i in range(num_weights)]
    values = [random.randint(10,50) for i in range(num_weights)]
    #weights = [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36]
    #values = [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25]
    sum_weights = sum(weights)
    data['weights'] = weights
    data['values'] = values
    data['items'] = list(range(len(weights)))
    data['num_items'] = len(weights)
    #num_bins = 5
    data['bins'] = list(range(num_bins))
    # all the bins have the same capacity, but, in general, it may be different
    data['bin_capacities'] = [int(sum_weights/1.2) for i in range(num_bins)]
    return data

data = create_data_model(5,2)

In [23]:
# create one model instance, with a name
m = Model(name='AMC Multiple Bin Problem')

# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[i, j] = m.binary_var(name='x_item_%i_to_bin_%i'%(i,j))

        
# Constraints
# Each item can be placed in at most one bin. 
#This constraint is set by requiring the sum of x[i][j] over all bins j to be less than or equal to 1.
for i in data['items']:
    m.add_constraint(m.sum([x[i, j] for j in data['bins']]) <= 1)

# The total weight packed in each bin can't exceed its capacity. 
#This constraint is set by requiring the sum of the weights of items placed in bin j to be less than or equal to the capacity of the bin.
for j in data['bins']:
    m.add_constraint(m.sum([x[i, j] * data['weights'][i] for i in data['items']]) <= data['bin_capacities'][j])

# Objective
# x(i, j)] * data['values'[i] adds the value of item i to the objective if the item is placed in bin j. 
#If i is not placed in any bin, its value doesn't contribute to the objective.
expr = LinearExpr(m)
objective_terms = []
for i in data['items']:
    for j in data['bins']:
        expr.add(x[i,j] * data['values'][i])
    
m.maximize(expr)

print(m.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Multiple Bin Problem

Maximize
 obj: 39 x_item_0_to_bin_0 + 39 x_item_0_to_bin_1 + 43 x_item_1_to_bin_0
      + 43 x_item_1_to_bin_1 + 37 x_item_2_to_bin_0 + 37 x_item_2_to_bin_1
      + 10 x_item_3_to_bin_0 + 10 x_item_3_to_bin_1 + 44 x_item_4_to_bin_0
      + 44 x_item_4_to_bin_1
Subject To
 c1: x_item_0_to_bin_0 + x_item_0_to_bin_1 <= 1
 c2: x_item_1_to_bin_0 + x_item_1_to_bin_1 <= 1
 c3: x_item_2_to_bin_0 + x_item_2_to_bin_1 <= 1
 c4: x_item_3_to_bin_0 + x_item_3_to_bin_1 <= 1
 c5: x_item_4_to_bin_0 + x_item_4_to_bin_1 <= 1
 c6: 32 x_item_0_to_bin_0 + 42 x_item_1_to_bin_0 + 37 x_item_2_to_bin_0
     + 45 x_item_3_to_bin_0 + 33 x_item_4_to_bin_0 <= 157
 c7: 32 x_item_0_to_bin_1 + 42 x_item_1_to_bin_1 + 37 x_item_2_to_bin_1
     + 45 x_item_3_to_bin_1 + 33 x_item_4_to_bin_1 <= 157

Bounds
 0 <= x_item_0_to_bin_0 <= 1
 0 <= x_item_0_to_bin_1 <= 1
 0 <= x_item_1_to_bin_0 <= 1
 0 <= x_item_1_to_bin_1 <= 1

In [24]:
result = m.solve

In [25]:
print(result())

solution for: AMC Multiple Bin Problem
objective: 173
x_item_0_to_bin_0=1
x_item_1_to_bin_0=1
x_item_2_to_bin_0=1
x_item_3_to_bin_0=1
x_item_4_to_bin_1=1



In [26]:
print('Total packed value:', result().get_objective_value())
total_weight = 0
for j in data['bins']:
    bin_weight = 0
    bin_value = 0
    print('Bin ', j, '\n')
    for i in data['items']:
        if result().get_value('x_item_%i_to_bin_%i'%(i,j)) > 0:
            print('Item', i, '- weight:', data['weights'][i], ' value:',
                  data['values'][i])
            bin_weight += data['weights'][i]
            bin_value += data['values'][i]
    print('Packed bin weight:', bin_weight)
    print('Packed bin value:', bin_value)
    print()
    total_weight += bin_weight
print('Total packed weight:', total_weight)

Total packed value: 173.0
Bin  0 

Item 0 - weight: 32  value: 39
Item 1 - weight: 42  value: 43
Item 2 - weight: 37  value: 37
Item 3 - weight: 45  value: 10
Packed bin weight: 156
Packed bin value: 129

Bin  1 

Item 4 - weight: 33  value: 44
Packed bin weight: 33
Packed bin value: 44

Total packed weight: 189


In [27]:
m.export_as_lp(path='502-Bin Packing Problem.lp')

'502-Bin Packing Problem.lp'

In [20]:
m.print_information()

Model: AMC Multiple Bin Problem
 - number of variables: 10
   - binary=10, integer=0, continuous=0
 - number of constraints: 7
   - linear=7
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
